In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
train_data = pd.read_json('./data/train.json')
test_data = pd.read_json('./data/test.json')

print(train_data.head())
print(train_data.shape)
print(test_data.head())
print(test_data.shape)


                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  
(1604, 5)
                                              band_1  \
0  [-15.863251, -15.201077, -17.887735, -19.

In [3]:
# Training data
x_band1_train = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in train_data["band_1"]])
x_band2_train = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in train_data["band_2"]])


# Testing data
x_band1_test = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in test_data["band_1"]])
x_band2_test = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in test_data["band_2"]])


In [ ]:
def residue_1(img,eps = 0.1, conn='4'):
    matlab_arr = matlab.double(img.tolist())
    matlab_imfs, matlab_residue, matlab_medel = eng.IEMD_public(matlab_arr, eps, 1, conn, nargout=3) 
    return np.asarray(matlab_residue).astype(np.float32).reshape(75,75,1)

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

x_band3 = np.array([residue_1(x_band1_train[i]) for i in range(x_band1_train.shape[0])])


In [ ]:
x_band1[i]

In [ ]:
X_train = np.concatenate([x_band1, x_band2], axis=-1)
Y_train = np.array(train_data["is_iceberg"])

X_test = np.concatenate([x_band1, x_band2], axis=-1)


In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras import regularizers
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization



# Create model
model = Sequential()
model.add(SeparableConv2D(32, 3, input_shape= activation="relu",depthwise_regularizer=regularizers.l2(0.01), depth_multiplier = 2))
model.add(BatchNormalization())
model.add(SeparableConv2D(64, 3, activation="relu",depthwise_regularizer=regularizers.l2(0.01), depth_multiplier = 1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(SeparableConv2D(32, 3, activation="relu",depthwise_regularizer=regularizers.l2(0.01), depth_multiplier = 2))
model.add(BatchNormalization())
model.add(SeparableConv2D(64, 3, activation="relu",depthwise_regularizer=regularizers.l2(0.01), depth_multiplier = 1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1, activation="sigmoid"))
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.summary()



In [ ]:

# Run model
model.fit(X_train, Y_train,validation_split=0.2)
score = model.evaluate(X_valid, Y_valid, batch_size=32,verbose = 1)
print(score)

In [ ]:
print(model.predict(X_train[0:10,:,:,:]))
print(Y_train[0:10])

In [ ]:
prediction = model.predict(X_test, verbose=1)
submit_df = pd.DataFrame({'id': test_data["id"], 'is_iceberg': prediction.flatten()})
submit_df.head()

In [ ]:
submit_df.to_csv("./naive_submission.csv", index=False)